In [ ]:
import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pandas as pd
import numpy as np
import string
from string import digits
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import tensorflow as tf

In [ ]:
data=pd.read_csv("../content/Hindi_English_Truncated_Corpus.csv",encoding='utf-8',names=['source','lang','trans','len_lang_sent','len_trans_sent'])

In [ ]:
data=data.iloc[1:,:]
data.head()

,source,lang,trans,len_lang_sent,len_trans_sent
1,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर...",NaN,NaN
2,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...,NaN,NaN
3,indic2012,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।,NaN,NaN
4,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते,NaN,NaN
5,indic2012,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।,NaN,NaN


In [ ]:
data=data[data.source=='ted']

In [ ]:
pd.isnull(data).sum()

source                0
lang                  0
trans                 0
len_lang_sent     29145
len_trans_sent    29145
dtype: int64

In [ ]:
data=data[~pd.isnull(data.trans)]
data.shape

(29145, 5)

In [ ]:
data.drop_duplicates(inplace=True)
data.drop(data[data.lang.str.contains(r'[0-9]')].index, inplace = True)
data.shape

(27040, 5)

In [ ]:
spchrs=set(string.punctuation)
lowerCase=lambda x:x.lower()
quotes=lambda x:re.sub("'",'',x)
specialCh=lambda x:''.join(ch for ch in x if ch not in spchrs)
rmDigits=lambda x:x.translate(str.maketrans('','',digits))
spaces=lambda x:x.strip()
unwanted=lambda x:re.sub(" +"," ",x)
startEnd=lambda x:'START_ '+x+' _END'
data.lang=data.lang.apply(quotes)
data.lang=data.lang.apply(specialCh)
data.lang=data.lang.apply(rmDigits)
data.lang=data.lang.apply(spaces)
data.lang=data.lang.apply(unwanted)
data.lang=data.lang.apply(lowerCase)
data.trans=data.trans.apply(quotes)
data.trans=data.trans.apply(specialCh)
data.trans=data.trans.apply(rmDigits)
data.trans=data.trans.apply(spaces)
data.trans=data.trans.apply(unwanted)
data.trans=data.trans.apply(lowerCase)
data.trans=data.trans.apply(startEnd)

In [ ]:
data.head()

,source,lang,trans,len_lang_sent,len_trans_sent
1,ted,politicians do not have permission to do what ...,START_ राजनीतिज्ञों के पास जो कार्य करना चाहिए...,NaN,NaN
2,ted,id like to tell you about one such child,START_ मई आपको ऐसे ही एक बच्चे के बारे में बता...,NaN,NaN
4,ted,what we really mean is that theyre bad at not ...,START_ हम ये नहीं कहना चाहते कि वो ध्यान नहीं ...,NaN,NaN
8,ted,and who are we to say even that they are wrong,START_ और हम होते कौन हैं यह कहने भी वाले कि व...,NaN,NaN
14,ted,so there is some sort of justice,START_ तो वहाँ न्याय है _END,NaN,NaN


In [ ]:
all_lang_words=set()
for line in data.lang:
    for word in line.split():
        if word not in all_lang_words:
            all_lang_words.add(word)

all_trans_words=set()
for line in data.trans:
    for word in line.split():
        if word not in all_trans_words:
            all_trans_words.add(word)
print(len(all_lang_words),len(all_trans_words))

14578 18418


In [ ]:
import pickle
with open('eng_words.pickle','wb') as handle:
    pickle.dump(all_lang_words,handle)
with open('hindi_words.pickle','wb') as handle:
    pickle.dump(all_trans_words,handle)

In [ ]:
data.head()

,source,lang,trans,len_lang_sent,len_trans_sent
1,ted,politicians do not have permission to do what ...,START_ राजनीतिज्ञों के पास जो कार्य करना चाहिए...,NaN,NaN
2,ted,id like to tell you about one such child,START_ मई आपको ऐसे ही एक बच्चे के बारे में बता...,NaN,NaN
4,ted,what we really mean is that theyre bad at not ...,START_ हम ये नहीं कहना चाहते कि वो ध्यान नहीं ...,NaN,NaN
8,ted,and who are we to say even that they are wrong,START_ और हम होते कौन हैं यह कहने भी वाले कि व...,NaN,NaN
14,ted,so there is some sort of justice,START_ तो वहाँ न्याय है _END,NaN,NaN


In [ ]:
data.len_lang_sent=data.lang.apply(lambda x:len(x.split(" ")))
data.len_trans_sent=data.trans.apply(lambda x:len(x.split(" ")))

In [ ]:
data=data[data.len_lang_sent<=20]
data=data[data.len_trans_sent<=20]
data.shape

(26809, 5)

In [ ]:
print("maximum length of lang Sentence ",max(data.len_lang_sent))
print("maximum length of trans Sentence ",max(data.len_trans_sent))

maximum length of lang Sentence  20
maximum length of trans Sentence  20


In [ ]:
max_length_src=max(data.len_lang_sent)
max_length_tar=max(data.len_trans_sent)
input_words = sorted(list(all_lang_words))
target_words = sorted(list(all_trans_words))
num_encoder_tokens = len(all_lang_words)
num_decoder_tokens = len(all_trans_words)
num_encoder_tokens, num_decoder_tokens

(14578, 18418)

In [ ]:
num_decoder_tokens+=1

In [ ]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [ ]:
data = shuffle(data)
data.head(10)

,source,lang,trans,len_lang_sent,len_trans_sent
74667,ted,perhaps this is just me maybe you,START_ शायद सिर्फ़ मैं ही या शायद आप भी _END,7,10
91750,ted,after referring to the black africans,START_ अश्वेत अफ़्रीकावासियों के लिए वह लिखता ...,6,9
41612,ted,and then you see this sort of model area aroun...,START_ अब बाहरी भाग के इस मॉडल जैसी जगह को देख...,12,12
61252,ted,in new york state robert moses spearheaded an ...,START_ न्यूयार्क में राबर्ट मोसेस ने राजमार्ग ...,12,13
13427,ted,who come through the village and have a meal i...,START_ जो उसके गाँव से निकलते है और उसके घर मे...,12,15
91578,ted,and it depended on if you were attacked,START_ और ये निर्भर करते हैं कि अगर आप पर हमला...,8,12
23741,ted,the unique design of the eiffel tower,START_ की अद्वितीय रचना के लिए प्रेरित किया _END,7,9
30324,ted,when we used to hang clothes on clotheslines o...,START_ जब हम घर के बाहर कपडे सुखा रहे होते थे ...,9,13
34326,ted,“would you praise me this way”,START_ “क्या तुम इस तरह मेरी तारीफ करोगे” _END,6,9
62950,ted,and the reason is that tool use,START_ और वह कारण यह है कि प्रारंभ से लेकर _END,7,11


In [ ]:
X, y = data.lang, data.trans
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((21447,), (5362,))

In [ ]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [ ]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word]
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] 
                    if t>0:
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [ ]:
latent_dim=256
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

In [ ]:
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 256)    3731968     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 256)    4715264     ['input_2[0][0]']                
                                                                                              

In [ ]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size =32
epochs =10

In [ ]:
model.fit(generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

Epoch 1/10
670/670 [==============================] - 76s 91ms/step - loss: 2.8665 - accuracy: 0.1532 - val_loss: 2.6826 - val_accuracy: 0.1897
Epoch 2/10
670/670 [==============================] - 58s 86ms/step - loss: 2.5700 - accuracy: 0.2072 - val_loss: 2.5795 - val_accuracy: 0.2193
Epoch 3/10
670/670 [==============================] - 58s 86ms/step - loss: 2.4245 - accuracy: 0.2368 - val_loss: 2.5284 - val_accuracy: 0.2417
Epoch 4/10
670/670 [==============================] - 58s 87ms/step - loss: 2.3147 - accuracy: 0.2598 - val_loss: 2.5014 - val_accuracy: 0.2542
Epoch 5/10
670/670 [==============================] - 60s 89ms/step - loss: 2.2249 - accuracy: 0.2785 - val_loss: 2.4860 - val_accuracy: 0.2611
Epoch 6/10
670/670 [==============================] - 58s 87ms/step - loss: 2.1454 - accuracy: 0.2960 - val_loss: 2.4827 - val_accuracy: 0.2663
Epoch 7/10
670/670 [==============================] - 58s 86ms/step - loss: 2.0758 - accuracy: 0.3120 - val_loss: 2.4714 - val_accuracy:

In [ ]:
model.save_weights('eng_to_hindi.h5')

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) 

decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [ ]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = target_token_index['START_']
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence

In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: and then you have to accept all sorts of things
Actual Hindi Translation:  सारी तरह की बातें माननी पड़ेंगी 
Predicted Hindi Translation:  और अगर आप एक और बहुत से लोग देख रहे हैं 


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-5])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: and the fact that we have english is now becoming
Actual Hindi Translation:  और अब तथ्य यह है कि हमारा अंग्रेजी होना
Predicted Hindi Translation:  और यही है कि हम यहाँ पर हैं 
